In [80]:
import mne 
from glob import glob 
import os
import matplotlib
from openpyxl import load_workbook
import numpy as np

In [81]:
matplotlib.use('Qt5Agg') 

In [82]:
#quanti soggetti ho nella mia cartella?
subs_file_path=glob("..\..\DATASET PRINCIPALE\CartellaEEG/*.vhdr")
print(len(subs_file_path))

2


In [83]:
#creo vettore file excel---- !ATTENZIONE FAR RUNNARE IL FILE EXCEL.YPTR per aggiustare gli excel
excel_file_path=glob("..\..\DATASET PRINCIPALE\Cartellaexcel/*.xlsx")
print(len(excel_file_path))

2


In [96]:
#Creo la funzione che legge e processa i dati 
def prep_data(file_path):
    data= mne.io.read_raw_brainvision(file_path, preload=True)
    data.set_eeg_reference();
    data.notch_filter(np.arange(50,100,50), filter_length='auto',phase='zero')
    data.notch_filter(np.arange(20,40,20), filter_length='auto',phase='zero')
    data.notch_filter(np.arange(10,20,10), filter_length='auto',phase='zero')
    data.filter(l_freq=0.1,h_freq=100)
    data.resample(250)
    return data

#Creo la funzione che estrae la matrice delle annotazioni dai file excel 
def get_vectors(data_path):

    book=load_workbook(data_path)
    sheet=book.active
    
    #onset
    onset=[]
    for row in sheet['a2':'a22249']:
        for cel in row:
               onset.append(cel.value)
                
    onset=np.array(onset)
    onset=onset*0.001
    
    #duration 
    duration=np.ones((22248,), dtype=int)*0.01
    
    #description 
    animal=['animal']
    description=[]
    for row in sheet ['e2':'e22249']:
        for cel in row:
            if(cel.value in animal):
                description.append(cel.value)
            else: 
                description.append('E 1')
    return [onset, duration, description] 

#creo la funzione che applica le annotazioni 
def fix_events(data_vector, matrix):
    myannot=[mne.Annotations(onset=matrix[0][0], duration=matrix[0][1],description=matrix[0][2])for i in range(len(excel_file_path))]
    for i in range(len(excel_file_path)):
        data_vector[i].set_annotations(myannot[i])
    
    return data_vector

In [97]:
#funzione preprocessing
prep_data_sub_vector= [prep_data(i)for i in subs_file_path]

Extracting parameters from ..\..\DATASET PRINCIPALE\CartellaEEG\sub-01_task-rsvp_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 3035739  =      0.000 ...  3035.739 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
Setting up band-stop filter from 19 - 21 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 19.45
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 19.20 Hz)
- Upper passband edge: 20.55 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 20.80 Hz)
- Filter length: 6601 samples (6.601 sec)

Setting up band-stop filter from 9.5 - 11 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (fir

In [98]:
#estraggo la matrice delle annotazioni 
matrix=[]
for i in range(len(excel_file_path)):
    matrix=[get_vectors(i)for i in excel_file_path]

In [99]:
data_vector_ok= fix_events(prep_data_sub_vector, matrix)

In [100]:
#ICA

#applico la ica per visualizzare i picchi di potenza da rimuovere
ica = mne.preprocessing.ICA(n_components=20, random_state=0)
ica.fit(data_vector_ok[0].filter(8, 35))
ica.plot_components(outlines="skirt")

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 413 samples (1.652 sec)



<ipython-input-100-9c9b70f80a6e>:4: DeprecationWarning: Version 0.23 introduced max_iter="auto", setting max_iter=1000 for `fastica` and max_iter=500 for `infomax` and `picard`. The current default of max_iter=200 will be changed to "auto" in version 0.24.
  ica = mne.preprocessing.ICA(n_components=20, random_state=0)


Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 20 components
Fitting ICA took 15.3s.


[<MNEFigure size 1950x1784 with 20 Axes>]

In [101]:
#escludo le componenti di rumore (viste dal grafico plottato)!!!!CONTROLLARLE OGNI VOLTA 
ica.exclude = [0,2,3,4,5,7,8,9,10,11,14,16,18,19]


In [102]:
events=mne.events_from_annotations(data_vector_ok[0])
#EPOCHS 
tmin=-0.100 #partono 100 millisecondi prima di un evento 
tmax=1 #finiscon 1000 milli secondo dopo 
baseline=(None,0)
epochs=mne.Epochs(data_vector_ok[0],events=events[0],tmin=tmin,tmax=tmax,baseline=baseline)

Used Annotations descriptions: ['E 1', 'animal']
Not setting metadata
Not setting metadata
22248 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated


In [103]:
evoked_animali=epochs['10002'].average()
evoked_altro=epochs['10001'].average()

In [104]:
evoked_animali.plot(spatial_colors=True)
evoked_altro.plot(spatial_colors=True)

<Figure size 2736x1616 with 2 Axes>